In [3]:
from utils import get_dataframes

dataframes, effect_wheight = get_dataframes("", True, False)

In [6]:
import pandas as pd
from utils import combined_damage_statistics


def damage_calculation(row: pd.Series):
    damages = row['damage']
    min_dmg, max_dmg, avg_dmg, std_dmg = combined_damage_statistics(damages)
    return min_dmg, max_dmg, avg_dmg, std_dmg


weapons_df = dataframes["weapons"]
weapons_df[["min_dmg", "max_dmg", "avg_dmg", "std_dmg"]] = weapons_df.apply(
    damage_calculation,
    axis=1,
    result_type="expand"
)

# weapons_df.sort_values(by=["weapon_type", "avg_dmg"])[["name", "weapon_type", "avg_dmg"]]
weapons_df

,name,weapon_type,prerequisites,range_,range_far,damage,abilities,min_dmg,max_dmg,avg_dmg,std_dmg
0,Banditen-Wurfbeilpaar,Äxte,"[[15, Str], [13, Dex]]",5,40,"[[1d8+4, slashing], [1d8+6, slashing]]","[Duo-Waffe, Zweihändig]",12.0,26.0,19.0,3.240370
1,Blutdolch der Sehnsüchte,Dolche,"[[15, Dex]]",5,30,"[[1d8, piercing], [2d8, acid]]",[Doppelter Angriff],3.0,24.0,13.5,3.968627
2,Blutstahlritter-Großschwert,Großschwerter,"[[16, Str], [11, Dex]]",10,0,"[[3d12, slashing], [2d4, thunder]]",[Zweihändig],5.0,44.0,24.5,6.184658
3,Bogen des Morgenrots,Kurzbögen,"[[16, Dex]]",55,80,"[[3d8, piercing], [2d10, fire]]","[Strahl des Morgenrots, Kraft des Morgenrots]",5.0,44.0,24.5,5.678908
4,Doppelschussbogen,Kurzbögen,"[[15, Dex]]",60,90,"[[2d8+2, piercing]]","[Zweihändig, Doppelschuss]",4.0,18.0,11.0,3.240370
5,Dornenbogen,Kurzbögen,"[[14, Dex]]",50,80,"[[2d12+4, piercing], [3d4, piercing]]","[Zweihändig, Dornenrankenexplosion]",9.0,40.0,24.5,5.251984
6,Drachenschlächtergroßaxt,Großäxte,"[[16, Str]]",5,0,"[[2d12, slashing], [4d4, lightning]]","[Zweihändig, Drachenblitz]",6.0,40.0,23.0,5.369668
7,Einfacher Dolch,Dolche,"[[10, Dex]]",5,30,"[[1d4, piercing]]",[Doppelter Angriff],1.0,4.0,2.5,1.118034
8,Elfisches Silberjagdmesser,Dolche,"[[13, Dex]]",5,35,"[[1d4, piercing], [1d8, psychic]]","[Lautloser Wanderer, Doppelter Angriff]",2.0,12.0,7.0,2.549510
9,Erzebrechender Zwergenzweihänder,Großschwerter,"[[13, Str], [12, Dex]]",10,0,"[[2d10, slashing], [2d6, bludgeoning]]",[Zweihändig],4.0,32.0,18.0,4.725816


In [15]:
import re

weapon_abilities_df = dataframes["weapon_abilities"]

patterns = {
    "extra_damage": r"(zusätzlichen Schaden|Schaden von \d+W\d+|Schadenswurf)",  # Detects extra damage mentions
    "double_attack": r"(doppelt angegriffen|zwei Treffer|zwei Geschosse)",  # Detects double attack ability
    "paralyze": r"(paralysiert|gelähmt)",  # Detects paralysis effects
    "bonus_action": r"(Bonusaktion|zusätzliche Aktion)",  # Detects bonus action usage
    "range_bonus": r"(Reichweite\s*\+\d+|weiter schießen|längere Reichweite)",  # Detects range enhancements
    "area_effect": r"(Explosion|rotierender Angriff|mehrere Ziele)",  # Detects area-based effects
    "exhausted": r"(erschöpft|Ermüdung|Ausdauerverlust|Kraftverlust)",
    "paralysed": r"(paralysiert|gelähmt|Bewegungsunfähig)",
    "petrified": r"(versteinert|zu Stein verwandelt)",
    "bleeding": r"(Blutung|blutet|schwere Wunde|ausbluten)",
    "knockback": r"(\d+ft. zurück|zurückgestoßen|zurückgedrängt)",
    "disadvantage_(hit)": r"(Nachteil auf Angriff|Angriffe haben Nachteil)",
    "disadvantage_(dmg)": r"(Nachteil auf Schaden|Schaden verringert)",
    "stunned": r"(betäubt|benommen|Handlungsunfähig)",
    "poisoned": r"(vergiftet|Giftwirkung|Giftresistenz erforderlich)",
    "grappled": r"(gepackt|umklammert|festgehalten)",
    "prone": r"(zu Boden geworfen|liegt am Boden|hingeworfen)",
    "double_damage": r"(doppelter Schaden|Schaden x2|Schadensmultiplikator)",
    "heal": r"(heilen|stellt \d+ Lebenspunkte wieder her|Heilwirkung)",
    "leap": r"(Sprung|springt über|kann weiter springen)",
    "frightened": r"(verängstigt|erschrocken|Fluchtreflex|Furcht)",
    "blinded": r"(geblendet|kann nicht sehen|blind)",
    "pull": r"(zieht das Ziel heran|heranziehen|zieht Gegner näher)",
    "increased_movement": r"(Bewegungsgeschwindigkeit erhöht|schneller bewegen|mehr Reichweite)",
    "escape": r"(kann entkommen|leicht flüchten|Fluchtmöglichkeit)",
    "invisible": r"(unsichtbar|nicht sichtbar|getarnt)",
    "restrained": r"(bewegungsunfähig gemacht|gefesselt|unbeweglich)",
    "resistance_(all)": r"(Resistenz gegen alle Schäden|reduzierter Schaden|Schutz vor Schaden)",
    "difficult_terrain": r"(erschwertes Gelände|langsamer bewegen|schwierige Fortbewegung)"
}


def extract_effects(description):
    effects = {key: bool(re.search(pattern, description, re.IGNORECASE)) for key, pattern in patterns.items()}
    return effects


effect_numerical = weapon_abilities_df["description"].apply(extract_effects).apply(pd.Series)
weapon_abilities_df_prepared = pd.concat([weapon_abilities_df, effect_numerical], axis=1)

weapon_abilities_df_prepared

,name,description,extra_damage,double_attack,paralyze,bonus_action,range_bonus,area_effect,exhausted,paralysed,...,leap,frightened,blinded,pull,increased_movement,escape,invisible,restrained,resistance_(all),difficult_terrain
0,Deckenbrecher,"Gegen unbelebte Objekte wie Türen, Kisten oder...",True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Doppelschuss,Mit dieser Waffe können zwei Geschosse gleichz...,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Doppelter Angriff,Mit der betreffenden Waffe darf doppelt angegr...,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Dornenrankenexplosion,,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Drachenblitz,,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,Duo-Waffe,Diese Waffe besteht aus zwei Teilen. Mit diese...,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,Herbstwinde,"Ein rotierender Angriff pro Rast, der Ziele 5f...",False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
7,Kraft des Morgenrots,Unter Einstrahlung von Tageslicht richtet der ...,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,Lautloser Wanderer,,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,Paralyse,Bei einem Treffer mit der Waffe wird ein d100 ...,False,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [16]:


weapon_abilities_df["ability_score"] = weapon_abilities_df_prepared[list(effect_weights.keys())].mul(
    pd.Series(effect_weights)).sum(axis=1)
weapon_abilities_df["strength_metric"] = (weapon_abilities_df["avg_damage"] * 0.5
                                          + weapon_abilities_df["range"] * 0.2
                                          + weapon_abilities_df["range_far"]
                                          + weapon_abilities_df["ability_score"] * 0.3)



NameError: name 'df' is not defined

In [11]:
effect_data
effect_data.get(effect_name, 0)

{'exhausted': 5,
 'paralysed': 5,
 'petrified': 5,
 'bleeding': 5,
 'knockback': 3,
 'disadvantage_(hit)': 4,
 'disadvantage_(dmg)': 4,
 'stunned': 4,
 'poisoned': 3,
 'grappled': 3,
 'prone': 3,
 'double_damage': 4,
 'heal': 2,
 'leap': 2,
 'frightened': 3,
 'blinded': 3,
 'pull': 2,
 'increased_movement': 3,
 'escape': 3,
 'invisible': 4,
 'restrained': 4,
 'resistance_(all)': 4,
 'difficult_terrain': 2}

In [3]:
def find_strongest_weapon(weapons_df: pd.DataFrame, user_attributes: list[tuple[str, int]],
                          prio_weapon_type: list[str]):
    user_attr_dict = {attr[0].lower(): attr[1] for attr in user_attributes}

    def meets_prerequisites(row):
        prerequisites = row['prerequisites']
        for value, attribute in prerequisites:
            if user_attr_dict.get(attribute.lower(), 0) < int(value):
                return False
        return True

    qualified_weapons = weapons_df[weapons_df.apply(meets_prerequisites, axis=1)]

    if prio_weapon_type:
        prio_weapon_type = [wtype.strip().lower() for wtype in prio_weapon_type]
        qualified_weapons = qualified_weapons.copy()  # Avoid chained assignment issues
        qualified_weapons.loc[:, 'weapon_type'] = qualified_weapons['weapon_type'].str.strip().str.lower()
        qualified_weapons = qualified_weapons[qualified_weapons['weapon_type'].isin(prio_weapon_type)]

    if not qualified_weapons.empty:
        best_weapon = qualified_weapons.loc[qualified_weapons['avg_dmg'].idxmax()]
        return best_weapon[['name', 'weapon_type', 'avg_dmg']].to_dict()
    else:
        return "No weapons match your qualifications."


user_stats = [["str", 14], ["dex", 17]]
prio_weapon_types = ["Dolche", "Stichschwerter"]
best_weapon = find_strongest_weapon(weapons_df, user_stats, prio_weapon_types)
best_weapon


{'name': 'Blutdolch der Sehnsüchte', 'weapon_type': 'dolche', 'avg_dmg': 13.5}

In [20]:
import re
import pandas as pd

# Load weapon and ability data
weapons_df = dataframes["weapons"]
weapon_abilities_df = dataframes["weapon_abilities"]


### STEP 1: Compute Damage Statistics ###
def damage_calculation(row: pd.Series):
    """
    Computes min, max, avg, and std deviation for weapon damage.
    """
    damages = row['damage']
    min_dmg, max_dmg, avg_dmg, std_dmg = combined_damage_statistics(damages)
    return min_dmg, max_dmg, avg_dmg, std_dmg


# Compute damage statistics and store them as new columns
weapons_df[["min_dmg", "max_dmg", "avg_dmg", "std_dmg"]] = weapons_df.apply(
    damage_calculation,
    axis=1,
    result_type="expand"
)

### STEP 2: Define Patterns for Ability Extraction ###
effect_patterns = {
    "extra_damage": {"pattern": r"(zusätzlichen Schaden|Schaden von \d+W\d+|Schadenswurf)", "value": 3,
                     "damage": r"(\d+W\d+)"},
    "double_attack": {"pattern": r"(doppelt angegriffen|zwei Treffer|zwei Geschosse)", "value": 4},
    "paralysed": {"pattern": r"(paralysiert|gelähmt|Bewegungsunfähig)", "value": 5, "duration": r"(\d+)\s*Runden"},
    "bonus_action": {"pattern": r"(Bonusaktion|zusätzliche Aktion)", "value": 2},
    "range_bonus": {"pattern": r"(Reichweite\s*\+\d+|weiter schießen|längere Reichweite)", "value": 2,
                    "bonus": r"(\d+)ft"},
    "area_effect": {"pattern": r"(Explosion|rotierender Angriff|mehrere Ziele)", "value": 3},
    "exhausted": {"pattern": r"(erschöpft|Ermüdung|Ausdauerverlust|Kraftverlust)", "value": 5},
    "petrified": {"pattern": r"(versteinert|zu Stein verwandelt)", "value": 5},
    "bleeding": {"pattern": r"(Blutung|blutet|schwere Wunde|ausbluten)", "value": 5, "damage": r"(\d+W\d+)"},
    "knockback": {"pattern": r"(\d+ft. zurück|zurückgestoßen|zurückgedrängt)", "value": 3, "distance": r"(\d+)ft"},
    "disadvantage_(hit)": {"pattern": r"(Nachteil auf Angriff|Angriffe haben Nachteil)", "value": 4},
    "disadvantage_(dmg)": {"pattern": r"(Nachteil auf Schaden|Schaden verringert)", "value": 4},
    "stunned": {"pattern": r"(betäubt|benommen|Handlungsunfähig)", "value": 4, "duration": r"(\d+)\s*Runden"},
    "poisoned": {"pattern": r"(vergiftet|Giftwirkung|Giftresistenz erforderlich)", "value": 3,
                 "duration": r"(\d+)\s*Runden"},
    "grappled": {"pattern": r"(gepackt|umklammert|festgehalten)", "value": 3},
    "prone": {"pattern": r"(zu Boden geworfen|liegt am Boden|hingeworfen)", "value": 3},
    "double_damage": {"pattern": r"(doppelter Schaden|Schaden x2|Schadensmultiplikator)", "value": 4},
    "heal": {"pattern": r"(heilen|stellt \d+ Lebenspunkte wieder her|Heilwirkung)", "value": 2,
             "heal_amount": r"(\d+) HP"},
    "leap": {"pattern": r"(Sprung|springt über|kann weiter springen)", "value": 2, "bonus": r"(\d+)ft"},
    "frightened": {"pattern": r"(verängstigt|erschrocken|Fluchtreflex|Furcht)", "value": 3},
    "blinded": {"pattern": r"(geblendet|kann nicht sehen|blind)", "value": 3},
    "pull": {"pattern": r"(zieht das Ziel heran|heranziehen|zieht Gegner näher)", "value": 2, "distance": r"(\d+)ft"},
    "increased_movement": {"pattern": r"(Bewegungsgeschwindigkeit erhöht|schneller bewegen|mehr Reichweite)",
                           "value": 3, "bonus": r"(\d+)ft"},
    "escape": {"pattern": r"(kann entkommen|leicht flüchten|Fluchtmöglichkeit)", "value": 3},
    "invisible": {"pattern": r"(unsichtbar|nicht sichtbar|getarnt)", "value": 4, "duration": r"(\d+)\s*Runden"},
    "restrained": {"pattern": r"(bewegungsunfähig gemacht|gefesselt|unbeweglich)", "value": 4},
    "resistance_(all)": {"pattern": r"(Resistenz gegen alle Schäden|reduzierter Schaden|Schutz vor Schaden)",
                         "value": 4},
    "difficult_terrain": {"pattern": r"(erschwertes Gelände|langsamer bewegen|schwierige Fortbewegung)", "value": 2},
}


### STEP 3: Extract Effects from Weapon Abilities ###
def extract_effects(description):
    """
    Extracts ability effects and stores them in a structured dictionary.
    """
    effects = {}
    for effect, data in effect_patterns.items():
        match = re.search(data["pattern"], description, re.IGNORECASE) if isinstance(description, str) else None
        if match:
            effect_data = {"value": data["value"]}

            # Extract additional information if available
            for attr, regex in data.items():
                if attr != "pattern" and attr != "value":
                    extra_match = re.search(regex, description)
                    if extra_match:
                        effect_data[attr] = int(extra_match.group(1)) if extra_match.group(
                            1).isdigit() else extra_match.group(1)

            effects[effect] = effect_data
    return effects


# Apply function to extract structured effects
weapon_abilities_df["effects"] = weapon_abilities_df["description"].apply(
    lambda x: extract_effects(x) if isinstance(x, str) else {})

# Compute ability score based on extracted effects
weapon_abilities_df["ability_score"] = weapon_abilities_df["effects"].apply(
    lambda x: sum(effect["value"] for effect in x.values()))

### STEP 4: Merge Weapons with Their Abilities ###
# Explode weapon abilities for proper mapping
weapons_exploded = weapons_df.explode("abilities")

# Merge with abilities
merged_df = weapons_exploded.merge(
    weapon_abilities_df[["name", "effects", "ability_score"]],
    left_on="abilities", right_on="name", how="left"
).drop(columns=["name_y"])

# Re-aggregate weapons after merging abilities
merged_df = merged_df.groupby(["name_x"], as_index=False).agg({
    "min_dmg": "first",
    "max_dmg": "first",
    "avg_dmg": "first",
    "std_dmg": "first",
    "range_": "first",
    "range_far": "first",
    "damage": "first",
    "ability_score": "sum",
    "effects": lambda x: {k: v for d in x.dropna() for k, v in d.items()}
})

# Compute strength metric
merged_df["strength_metric"] = (
        merged_df["avg_dmg"] +
        merged_df["range_"] / 5 * 0.2 +
        (merged_df["range_far"] / 5) ** 1 / 2 * 0.2 +
        merged_df["ability_score"]
)

merged_df

,name_x,min_dmg,max_dmg,avg_dmg,std_dmg,range_,range_far,damage,ability_score,effects,strength_metric
0,Banditen-Wurfbeilpaar,12.0,26.0,19.0,3.240370,5,40,"[[1d8+4, slashing], [1d8+6, slashing]]",4.0,{'double_attack': {'value': 4}},24.0
1,Blutdolch der Sehnsüchte,3.0,24.0,13.5,3.968627,5,30,"[[1d8, piercing], [2d8, acid]]",4.0,{'double_attack': {'value': 4}},18.3
2,Blutstahlritter-Großschwert,5.0,44.0,24.5,6.184658,10,0,"[[3d12, slashing], [2d4, thunder]]",0.0,{},24.9
3,Bogen des Morgenrots,5.0,44.0,24.5,5.678908,55,80,"[[3d8, piercing], [2d10, fire]]",0.0,{},28.3
4,Doppelschussbogen,4.0,18.0,11.0,3.240370,60,90,"[[2d8+2, piercing]]",4.0,{'double_attack': {'value': 4}},19.2
5,Dornenbogen,9.0,40.0,24.5,5.251984,50,80,"[[2d12+4, piercing], [3d4, piercing]]",0.0,{},28.1
6,Drachenschlächtergroßaxt,6.0,40.0,23.0,5.369668,5,0,"[[2d12, slashing], [4d4, lightning]]",0.0,{},23.2
7,Einfacher Dolch,1.0,4.0,2.5,1.118034,5,30,"[[1d4, piercing]]",4.0,{'double_attack': {'value': 4}},7.3
8,Elfisches Silberjagdmesser,2.0,12.0,7.0,2.549510,5,35,"[[1d4, piercing], [1d8, psychic]]",4.0,{'double_attack': {'value': 4}},11.9
9,Erzebrechender Zwergenzweihänder,4.0,32.0,18.0,4.725816,10,0,"[[2d10, slashing], [2d6, bludgeoning]]",0.0,{},18.4
